In [301]:
import pandas as pd 
import numpy as np

In [302]:
doc = pd.read_csv('Dataset/CassiopéeNorm/Avion.csv')

In [303]:
def format(df):
    # On créé les nouvelles colonnes en combinant les valeurs des colonnes d'origines correspondant à l'articulation
    df['chestUpper']= df['chestUpper_T_glob'].str.cat(df['chestUpper_T_glob.1'], sep=',').str.cat(df['chestUpper_T_glob.2'], sep = ',')  
    df['rShldrBend']= df['rShldrBend_T_glob'].str.cat(df['rShldrBend_T_glob.1'], sep=',').str.cat(df['rShldrBend_T_glob.2'], sep = ',')
    df['lShldrBend']= df['lShldrBend_T_glob'].str.cat(df['lShldrBend_T_glob.1'], sep=',').str.cat(df['lShldrBend_T_glob.2'], sep = ',') 
    df['rForearmBend']= df['rForearmBend_T_glob'].str.cat(df['rForearmBend_T_glob.1'], sep=',').str.cat(df['rForearmBend_T_glob.2'], sep = ',')
    df['lForearmBend']= df['lForearmBend_T_glob'].str.cat(df['lForearmBend_T_glob.1'], sep=',').str.cat(df['lForearmBend_T_glob.2'], sep = ',')
    df['lHand']= df['lHand_T_glob'].str.cat(df['lHand_T_glob.1'], sep=',').str.cat(df['lHand_T_glob.2'], sep = ',')
    df['rHand']= df['rHand_T_glob'].str.cat(df['rHand_T_glob.1'], sep=',').str.cat(df['rHand_T_glob.2'], sep = ',')
    df['pelvis']= df['pelvis_T_glob'].str.cat(df['pelvis_T_glob.1'], sep=',').str.cat(df['pelvis_T_glob.2'], sep = ',')
    
    # On les réorganise
    df = df[['Frame', 'Time(s)', 'pelvis', 'chestUpper', 'lShldrBend', 'lForearmBend', 'lHand', 'rShldrBend', 'rForearmBend', 'rHand']]
    
    # On convertit les colonnes en tableau numpy afin de pouvoir séparer les valeurs des chaines de caractère dans des listes de coordonnées
    chestUpper = df['chestUpper'].values
    rShldrBend = df['rShldrBend'].values
    rForearmBend = df['rForearmBend'].values
    rHand = df['rHand'].values
    lShldrBend = df['lShldrBend'].values
    lForearmBend = df['lForearmBend'].values
    lHand = df['lHand'].values
    pelvis = df['pelvis'].values

    # On sépare les valeurs
    for i in range(1,df['chestUpper'].values.shape[0]):
        chestUpper[i] = chestUpper[i].split(',')
        rShldrBend[i] = rShldrBend[i].split(',')
        rForearmBend[i] = rForearmBend[i].split(',')
        rHand[i] = rHand[i].split(',')
        lShldrBend[i] = lShldrBend[i].split(',')
        lForearmBend[i] = lForearmBend[i].split(',')
        lHand[i] = lHand[i].split(',')
        pelvis[i] = pelvis[i].split(',')
    
    #On met à jour le df et on le renvoie
    df['chestUpper']= chestUpper
    df['rShldrBend']= rShldrBend
    df['rForearmBend']=  rForearmBend
    df['rHand']= rHand
    df['lShldrBend']= lShldrBend
    df['lForearmBend']= lForearmBend
    df['lHand']= lHand
    df['pelvis']= pelvis
    df.drop([0], axis = 0, inplace = True)
    return df



In [304]:
def to_categorical(annotation_file):
    data = pd.read_csv(annotation_file, sep="\t")
    cols = list(data.columns)
    data.head()
    action_class, label = cols[0], cols[1]

    def label_change(x):
        if x == 'Mono':
            return 1
        elif x == 'Bi':
            return 1
        else:
            return 2

    data[label] = data[label].apply(label_change(label))
    data.to_csv(annotation_file, index=False)

In [305]:
import pandas as pd
import numpy as np
import struct
import codecs

def to_skeleton(df, name_df):
    joints_df = df.iloc[:, 2:]
    data = joints_df.to_numpy()

    num_joints = data.shape[1]
    num_frames = data.shape[0]
    body_info = [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2]

    with open('./Dataset/Skeletons_2/'+ '{}'.format(name_df) + '.skeleton', 'w') as f:

        f.write("{}\n".format(num_frames))
        
    

        for frame in data:
            f.write("1\n")
            f.write("{} {} {} {} {} {} {} {} {} {} {}\n".format(body_info[0], body_info[1], body_info[2], body_info[3], body_info[4], body_info[5], body_info[6], body_info[7], body_info[8], body_info[9], body_info[10]))
            f.write("{}\n".format(num_joints))
            for joint in range(num_joints):

                x = float(frame[joint][0])
                y = float(frame[joint][1])
                z = float(frame[joint][2])
            

                f.write("{} {} {}\n".format(x, y, z))

    f.close()


In [307]:
import os

annotation = pd.read_csv('Dataset/Annotation_gloses.csv', sep='\t') # On lit le fichier d'annotation

cols = list(annotation.columns) # On récupère une liste des noms des colonnes
action_class, label = cols[0], cols[1] # On récupère les noms des colonnes

for file_name in os.listdir('Dataset/CassiopéeNorm'): # On parcourt les fichiers du dataset brut de MocapLab

    num = annotation[annotation[action_class] == file_name[:-4]].index.values # On récupère l'index de la ligne du fichiers d'annotation correspondant au fichier du dataset brut
    if len(num) > 0:
        num = num[0]
        name_df = "S001C001P001A" + "{:03d}".format(num) # On crée le nom du fichier de sortie en fonction de l'index de la ligne
        to_skeleton(format(pd.read_csv('Dataset/CassiopéeNorm/' + file_name)), name_df) # On convertit le fichier du dataset brut en fichier skeleton


C:\Users\titit\AppData\Local\Temp\ipykernel_19200\3231791372.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chestUpper']= chestUpper
C:\Users\titit\AppData\Local\Temp\ipykernel_19200\3231791372.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rShldrBend']= rShldrBend
C:\Users\titit\AppData\Local\Temp\ipykernel_19200\3231791372.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c